In [1]:
%load_ext autoreload
%autoreload 2

In [138]:
import pandas as pd
import numpy as np
from WorkforceSentimentMonitoring.data import get_data, merge, holdout, drop_wrong_language

## Import & Clean Data

In [3]:
submission, train, test = get_data()
df = merge(submission, train, test)
df = drop_wrong_language(df, 'review')
target = [
    "work-balance",
    "culture-values",
    "career-opportunities",
    "comp-benefits",
    "senior-mgmt",
    "overall",
]

  0%|          | 0/10 [00:00<?, ?it/s]

Identifying entries in other languages...


100%|██████████| 10/10 [08:51<00:00, 53.17s/it]


Drop 434 entries? [y] / n

Dropping 434 entries...


### Deal with Contractions

In [9]:
import contractions

In [13]:
df.review[1]

'Moving at the speed of light, burn out is inevitable 1) Food, food, food. 15+ cafes on main campus (MTV) alone. Mini-kitchens, snacks, drinks, free breakfast/lunch/dinner, all day, errr\'day.  2) Benefits/perks. Free 24:7 gym access (on MTV campus). Free (self service) laundry (washer/dryer) available. Bowling alley. Volley ball pit. Custom-built and exclusive employee use only outdoor sport park (MTV). Free health/fitness assessments. Dog-friendly. Etc. etc. etc.  3) Compensation. In ~2010 or 2011, Google updated its compensation packages so that they were more competitive.  4) For the size of the organization (30K+), it has remained relatively innovative, nimble, and fast-paced and open with communication but, that is definitely changing (for the worse).  5) With so many departments, focus areas, and products, *in theory*, you should have plenty of opportunity to grow your career (horizontally or vertically). In practice, not true.  6) You get to work with some of the brightest, mos

In [16]:
def expand_contractions(text_series):
    return text_series.apply(contractions.fix)

In [17]:
df['review'] = expand_contractions(df['review'])

In [20]:
df.review[1]

'Moving at the speed of light, burn out is inevitable 1) Food, food, food. 15+ cafes on main campus (MTV) alone. Mini-kitchens, snacks, drinks, free breakfast/lunch/dinner, all day, errr\'day.  2) Benefits/perks. Free 24:7 gym access (on MTV campus). Free (self service) laundry (washer/dryer) available. Bowling alley. Volley ball pit. Custom-built and exclusive employee use only outdoor sport park (MTV). Free health/fitness assessments. Dog-friendly. Etc. etc. etc.  3) Compensation. In ~2010 or 2011, Google updated its compensation packages so that they were more competitive.  4) For the size of the organization (30K+), it has remained relatively innovative, nimble, and fast-paced and open with communication but, that is definitely changing (for the worse).  5) With so many departments, focus areas, and products, *in theory*, you should have plenty of opportunity to grow your career (horizontally or vertically). In practice, not true.  6) You get to work with some of the brightest, mos

### Preprocess

In [30]:
# Define X and y
X = pd.DataFrame(df.review)
y = df[target].copy()

In [31]:
from WorkforceSentimentMonitoring.encoders import Preprocessor

In [32]:
X

,review
0,Best Company to work for People are smart and ...
1,"Moving at the speed of light, burn out is inev..."
2,Great balance between big-company security and...
3,The best place I have worked and also the most...
4,Execellent for engineers Impact driven. Best t...
...,...
52376,great place to grow! Great health benefits. Ma...
52377,An ocean of opportunities diverse set of peopl...
52378,Tech Gaint Equip its employees wid huge salari...
52379,Terrible They had great health benefits (no lo...


In [33]:
preprocessor = Preprocessor()
preprocessor.fit_transform(X)

,review
0,best company to work for people are smart and ...
1,moving at the speed of light burn out is inev...
2,great balance between big company security and...
3,the best place i have worked and also the most...
4,execellent for engineer impact driven best te...
...,...
52376,great place to grow great health benefit man...
52377,an ocean of opportunity diverse set of people ...
52378,tech gaint equip it employee wid huge salary ...
52379,terrible they had great health benefit no lon...


In [53]:
X['review'] = X.review.str.replace('\s+', ' ')

In [72]:
X['review'] = X.review.str.strip()

## Import Lexicon

In [35]:
lexicon = pd.read_csv('../lexicon/EmotionIntensityLexicon.txt',sep='\t')

In [36]:
lexicon.emotion.unique()

array(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness',
       'surprise', 'trust'], dtype=object)

In [37]:
lexicon

,word,emotion,emotion-intensity-score
0,outraged,anger,0.964
1,brutality,anger,0.959
2,hatred,anger,0.953
3,hateful,anger,0.940
4,terrorize,anger,0.939
...,...,...,...
9916,fugitive,trust,0.141
9917,divorce,trust,0.133
9918,mistakes,trust,0.133
9919,bait,trust,0.133


## Bag of Words

In [74]:
tmp = X.review[1].split(' ')

In [75]:
tmp = [word for word in tmp if word]

In [76]:
from sklearn.feature_extraction.text import CountVectorizer

In [79]:
from WorkforceSentimentMonitoring.preprocessing import remove_stopwords

In [83]:
X.review = X.review.apply(remove_stopwords)

In [88]:
X['length'] = X.review.str.split(' ').apply(len)

In [92]:
vectorizer = CountVectorizer()

In [93]:
X_vectorized = vectorizer.fit_transform(X.review)

In [95]:
X_vectorized.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [99]:
X_vectorized = pd.DataFrame(X_vectorized.toarray(), columns=vectorizer.get_feature_names())

In [107]:
emo_words = [word for word in lexicon.word if word in X_vectorized.columns]

In [108]:
emo_words

['outraged',
 'brutality',
 'hatred',
 'hateful',
 'furious',
 'enraged',
 'furiously',
 'execution',
 'angered',
 'rage',
 'loathe',
 'hostile',
 'murder',
 'pissed',
 'explosive',
 'vengeful',
 'ferocious',
 'killing',
 'combative',
 'vengeance',
 'wrath',
 'torment',
 'vicious',
 'threatening',
 'abhorrent',
 'fighting',
 'attacking',
 'bloodshed',
 'assault',
 'assassination',
 'strangle',
 'explode',
 'malicious',
 'hostility',
 'attack',
 'hell',
 'murderous',
 'malice',
 'beating',
 'outrage',
 'irate',
 'tumultuous',
 'destroying',
 'violent',
 'stab',
 'slaughter',
 'abomination',
 'obliterate',
 'belligerent',
 'prick',
 'cruelty',
 'horrid',
 'rabid',
 'torture',
 'hate',
 'hating',
 'tyrannical',
 'demonic',
 'despicable',
 'angry',
 'livid',
 'madman',
 'vindictive',
 'terrorist',
 'venomous',
 'threaten',
 'savage',
 'atrocity',
 'fierce',
 'abuse',
 'tyrant',
 'anger',
 'slam',
 'punching',
 'punched',
 'destructive',
 'ruthless',
 'slap',
 'destroyed',
 'retaliatory',
 

In [109]:
X_vectorized[emo_words]

,outraged,brutality,hatred,hateful,furious,enraged,furiously,execution,angered,rage,...,deny,mislead,unaccountable,unreliable,addict,falsehood,thang,divorce,mistakes,bait
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52376,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52377,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52378,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
52379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
X

,review,length
0,best company work people smart friendly bureau...,9
1,moving speed light burn inevitable food food f...,389
2,great balance big company security fun fast mo...,440
3,best place worked also demanding find well reg...,387
4,execellent engineer impact driven best tech wo...,13
...,...,...
52376,great place grow great health benefit many int...,27
52377,ocean opportunity diverse set people problem s...,32
52378,tech gaint equip employee wid huge salary high...,23
52379,terrible great health benefit longer told many...,57


In [130]:
lexicon

,word,emotion,emotion-intensity-score
0,outraged,anger,0.964
1,brutality,anger,0.959
2,hatred,anger,0.953
3,hateful,anger,0.940
4,terrorize,anger,0.939
...,...,...,...
9916,fugitive,trust,0.141
9917,divorce,trust,0.133
9918,mistakes,trust,0.133
9919,bait,trust,0.133


In [150]:
table = pd.pivot_table(lexicon, values='emotion-intensity-score', index='word', columns='emotion', fill_value=0)

In [151]:
table

emotion,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
word,,,,,,,,
aaaaaaah,0.000,0.00,0.0,0.344,0.000,0.000,0.000,0.000
aaaah,0.000,0.00,0.0,0.234,0.000,0.000,0.000,0.000
abacus,0.000,0.00,0.0,0.000,0.000,0.000,0.000,0.406
abandon,0.000,0.00,0.0,0.531,0.000,0.703,0.000,0.000
abandoned,0.222,0.00,0.0,0.534,0.000,0.828,0.000,0.000
...,...,...,...,...,...,...,...,...
zany,0.000,0.00,0.0,0.000,0.000,0.000,0.555,0.000
zeal,0.000,0.50,0.0,0.000,0.547,0.000,0.484,0.328
zealous,0.000,0.00,0.0,0.000,0.393,0.000,0.000,0.258


In [152]:
table.loc[]

,review,length
0,best company work people smart friendly bureau...,9
1,moving speed light burn inevitable food food f...,389
2,great balance big company security fun fast mo...,440
3,best place worked also demanding find well reg...,387
4,execellent engineer impact driven best tech wo...,13
...,...,...
52376,great place grow great health benefit many int...,27
52377,ocean opportunity diverse set people problem s...,32
52378,tech gaint equip employee wid huge salary high...,23
52379,terrible great health benefit longer told many...,57


In [180]:
X_vectorized.loc[:, X_vectorized.loc[1] > 0]

absolutely    1
access        1
actually      4
adaptive      2
ago           1
             ..
working       1
worse         1
wow           1
year          1
yet           1
Name: 1, Length: 267, dtype: int64

In [181]:
lexicon

,word,emotion,emotion-intensity-score
0,outraged,anger,0.964
1,brutality,anger,0.959
2,hatred,anger,0.953
3,hateful,anger,0.940
4,terrorize,anger,0.939
...,...,...,...
9916,fugitive,trust,0.141
9917,divorce,trust,0.133
9918,mistakes,trust,0.133
9919,bait,trust,0.133


In [183]:
select_row = lexicon[(lexicon.word == 'hatred') & (lexicon.emotion == 'anger')]

In [194]:
select_row.iloc[0]['emotion-intensity-score']

0.953

In [196]:
def get_emotion_score(row, lexicon, emotion):
    review = row['review']
    review_words = review.split(' ')
    score = 0
    for word in review_words:
        select_row = lexicon[(lexicon.word == word) & (lexicon.emotion == emotion)]
        if len(select_row) > 0:
            score += select_row.iloc[0]['emotion-intensity-score']
    return score / row['length']

In [203]:
tmp = X.sample(100)

In [206]:
emotions = lexicon.emotion.unique()

In [207]:
emotions

array(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'sadness',
       'surprise', 'trust'], dtype=object)

In [209]:
tmp

,review,length
20316,welcome cloud operation came microsoft b ecomm...,110
6203,part time fulfillment great environment fun ha...,16
22493,director dynamic intense school many disciplin...,64
42244,time apple far great work environment best eve...,13
34057,great learning opportunity coupled unrealistic...,58
...,...,...
24187,seti perk benefit great many manager understan...,15
21244,sdet good source info good class bad managemen...,18
39347,research intern computational resource great c...,17
42913,good good atmosphere wa good great yea like ad...,9


In [212]:
from tqdm import tqdm

In [ ]:
for emotion in tqdm(emotions):
    tmp[f'{emotion}_score'] = tmp.apply(lambda x: get_emotion_score(x, lexicon, emotion), axis=1)

 12%|█▎        | 1/8 [00:13<01:32, 13.28s/it]

In [201]:
tmp['anger_score'] = tmp.apply(lambda x: get_emotion_score(x, lexicon, 'anger'), axis=1)

In [202]:
tmp

,review,length,anger_score
51833,great place start career great place start car...,12,0.000000
20726,microsoft lot product brings ton missing mobil...,13,0.054769
23366,great place family oriented people microsoft ...,39,0.026846
12776,dublin best facebook office freedom awesome pe...,27,0.000000
27853,worst company ever worked nothing good working...,43,0.079721
...,...,...,...
21205,good place work fun environment different reta...,26,0.000000
12320,fabulous employer great pay awesome benefit ta...,34,0.003000
16425,specialist consistent coaching across level ne...,13,0.000000
8707,sr recruiter working smart people ability infl...,34,0.000000
